In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
height = 28
width = 28
channels = 1
n_filters = 8

n_hidden1 = 500
n_hidden2 = 100
n_hidden3 = 50
n_output = 10
learning_rate = 0.1

X = tf.placeholder(tf.float32, shape=(None, height * width))
y = tf.placeholder(tf.int64, shape=(None), name="y")

input_layers = tf.reshape(X, [-1,height,width,channels])

convolution = tf.layers.conv2d(input_layers,filters=n_filters, kernel_size = [5,5], 
                           padding = "SAME", activation = tf.nn.relu)
pool = tf.layers.max_pooling2d(inputs=convolution, pool_size=[2, 2], strides=2)
pool_flat = tf.reshape(pool, [-1,n_filters * 14 * 14])


with tf.contrib.framework.arg_scope([fully_connected], activation_fn = tf.nn.relu):
    hidden1 = fully_connected(pool_flat,n_hidden1)
    hidden2 = fully_connected(hidden1,n_hidden2)
    hidden3 = fully_connected(hidden2,n_hidden3)
    logits = fully_connected(hidden3, n_output, activation_fn=None)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [4]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()


n_epochs = 10
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)


0 Train accuracy: 1.0 Test accuracy: 0.9749
1 Train accuracy: 1.0 Test accuracy: 0.9827
2 Train accuracy: 1.0 Test accuracy: 0.9864
3 Train accuracy: 1.0 Test accuracy: 0.987
4 Train accuracy: 1.0 Test accuracy: 0.9872
5 Train accuracy: 1.0 Test accuracy: 0.9877
6 Train accuracy: 1.0 Test accuracy: 0.9874
7 Train accuracy: 1.0 Test accuracy: 0.9854
8 Train accuracy: 1.0 Test accuracy: 0.9891
9 Train accuracy: 1.0 Test accuracy: 0.9894
